In [113]:
from IPython import display
import requests
import json
from sodapy import Socrata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
import time
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_columns',50)

from sklearn.preprocessing import OneHotEncoder


# Introduction

According to the CDC, more than 48 million Americans per year become sick from food, and an estimated 75% of the outbreaks came from food prepared by caterers, delis, and restaurants. In most cities, health inspections are generally random, which can increase time spent on spot checks at clean restaurants that have been following the rules closely — and missed opportunities to improve health and hygiene at places with more pressing food safety issues.

The goal for this project is to leverage public citizen generated data from social media to narrow the search for critical health and safety violations in New York City. As the City of New York manages  an open data portal, everyone can access historical hygiene inspections and violation records. By combine these two data source this project aims to determine which words, phrases, ratings, and patterns among restaurants lead to critical health and safety violations. This model can assist city health inspectors do their job better by prioritizing the kitchens most likely to be in violation of code.

# Obtain

This project requires data pulled from two different sources, the City of New York and Yelp. To obtain the data we will call the API keys.

## NYC Open Data Portal

In [114]:
doh = pd.read_csv('data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')
doh

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50008319,POULETTE,Manhattan,790,9 AVENUE,10019.0,2129569488,Chicken,01/28/2019,Violations were cited in the following area(s).,04D,"Food worker does not wash hands thoroughly after using the toilet, coughing, sneezing, smoking, ...",Critical,22,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.765040,-73.987795,104.0,3.0,13300.0,1025235.0,1.010430e+09,MN15
1,41678734,TINY'S DINER,Bronx,3603,RIVERDALE AVENUE,10463.0,7187087600,American,09/23/2019,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or non-food areas.,Critical,24,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.886691,-73.907056,208.0,11.0,29500.0,2084186.0,2.057960e+09,BX29
2,40795021,TAO RESTAURANT,Manhattan,42,EAST 58 STREET,10022.0,2128882288,Asian/Asian Fusion,03/11/2020,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,9,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.762786,-73.971486,105.0,4.0,11202.0,1036073.0,1.012930e+09,MN17
3,40795021,TAO RESTAURANT,Manhattan,42,EAST 58 STREET,10022.0,2128882288,Asian/Asian Fusion,03/11/2020,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,9,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.762786,-73.971486,105.0,4.0,11202.0,1036073.0,1.012930e+09,MN17
4,40538234,"DUNKIN',' BASKIN ROBBINS",Queens,9925,HORACE HARDING EXPRESSWAY,11368.0,7182719222,Donuts,07/29/2021,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-use food dispensing utensil, improperly used or sto...",Critical,10,A,07/29/2021,03/24/2022,Cycle Inspection / Initial Inspection,40.736185,-73.858052,404.0,21.0,43702.0,4047915.0,4.019480e+09,QN25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186222,41556790,FIVE GUYS FAMOUS BURGERS AND FRIES,Manhattan,2847,BROADFWAY,NaN,2126787701,Hamburgers,03/01/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,3,A,03/01/2019,03/24/2022,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000e+00,NaN
186223,50032876,TANNER SMITH'S,Manhattan,204,WEST 55 STREET,10019.0,9175172283,American,06/07/2019,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or conditions conducive to attracting vermin to the premise...,Not Critical,12,A,06/07/2019,03/24/2022,Cycle Inspection / Initial Inspection,40.764364,-73.981362,105.0,4.0,13700.0,1024857.0,1.010260e+09,MN17
186224,50003842,T- 45,Manhattan,135,WEST 45 STREET,10036.0,6466403775,American,07/26/2018,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not ...,Not Critical,49,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.757244,-73.983815,105.0,4.0,11900.0,1089796.0,1.009980e+09,MN17
186225,41640824,RICURAS ECUADORIAN BAKERY,Bronx,1576,WATSON AVENUE,10472.0,7184508363,Spanish,07/20/2018,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizing solution.,Critical,12,A,07/20/2018,03/24/2022,Cycle Inspection / Initial Inspection,40.826352,-73.876066,209.0,18.0,5001.0,2023533.0,2.037160e+09,BX55


In [115]:
doh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186227 entries, 0 to 186226
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  186227 non-null  int64  
 1   DBA                    186200 non-null  object 
 2   BORO                   186227 non-null  object 
 3   BUILDING               185799 non-null  object 
 4   STREET                 186227 non-null  object 
 5   ZIPCODE                183146 non-null  float64
 6   PHONE                  186214 non-null  object 
 7   CUISINE DESCRIPTION    186226 non-null  object 
 8   INSPECTION DATE        186227 non-null  object 
 9   ACTION                 186227 non-null  object 
 10  VIOLATION CODE         185413 non-null  object 
 11  VIOLATION DESCRIPTION  185863 non-null  object 
 12  CRITICAL FLAG          186227 non-null  object 
 13  SCORE                  186227 non-null  int64  
 14  GRADE                  67933 non-nul

In [123]:
doh.dropna(subset=['PHONE'],inplace=True)

In [124]:
doh.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186214 entries, 0 to 186226
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  186214 non-null  int64  
 1   DBA                    186187 non-null  object 
 2   BORO                   186214 non-null  object 
 3   BUILDING               185786 non-null  object 
 4   STREET                 186214 non-null  object 
 5   ZIPCODE                183133 non-null  float64
 6   PHONE                  186214 non-null  object 
 7   CUISINE DESCRIPTION    186213 non-null  object 
 8   INSPECTION DATE        186214 non-null  object 
 9   ACTION                 186214 non-null  object 
 10  VIOLATION CODE         185400 non-null  object 
 11  VIOLATION DESCRIPTION  185850 non-null  object 
 12  CRITICAL FLAG          186214 non-null  object 
 13  SCORE                  186214 non-null  int64  
 14  GRADE                  67924 non-nul

In [125]:
# How many unique restaurants are in this dataset?
doh['CAMIS'].nunique()

19790

Health code violations found during an inspections carries a point value, and a restaurant’s score corresponds to a letter grade. A lower point score, leads to a better letter grade:

"A" grade: 0 to 13 points for sanitary violations
"B" grade: 14 to 27 points for sanitary violations
"C" grade: 28 or more points for sanitary violations

In [126]:
doh['A'] = doh['SCORE'] < 14
doh['B'] = (doh['SCORE'] > 13) & (doh['SCORE'] < 28)
doh['C'] = doh['SCORE'] > 27

In [127]:
doh

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,A,B,C
0,50008319,POULETTE,Manhattan,790,9 AVENUE,10019.0,2129569488,Chicken,01/28/2019,Violations were cited in the following area(s).,04D,"Food worker does not wash hands thoroughly after using the toilet, coughing, sneezing, smoking, ...",Critical,22,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.765040,-73.987795,104.0,3.0,13300.0,1025235.0,1.010430e+09,MN15,False,True,False
1,41678734,TINY'S DINER,Bronx,3603,RIVERDALE AVENUE,10463.0,7187087600,American,09/23/2019,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or non-food areas.,Critical,24,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.886691,-73.907056,208.0,11.0,29500.0,2084186.0,2.057960e+09,BX29,False,True,False
2,40795021,TAO RESTAURANT,Manhattan,42,EAST 58 STREET,10022.0,2128882288,Asian/Asian Fusion,03/11/2020,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,9,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.762786,-73.971486,105.0,4.0,11202.0,1036073.0,1.012930e+09,MN17,True,False,False
3,40795021,TAO RESTAURANT,Manhattan,42,EAST 58 STREET,10022.0,2128882288,Asian/Asian Fusion,03/11/2020,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,9,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.762786,-73.971486,105.0,4.0,11202.0,1036073.0,1.012930e+09,MN17,True,False,False
4,40538234,"DUNKIN',' BASKIN ROBBINS",Queens,9925,HORACE HARDING EXPRESSWAY,11368.0,7182719222,Donuts,07/29/2021,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-use food dispensing utensil, improperly used or sto...",Critical,10,A,07/29/2021,03/24/2022,Cycle Inspection / Initial Inspection,40.736185,-73.858052,404.0,21.0,43702.0,4047915.0,4.019480e+09,QN25,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186222,41556790,FIVE GUYS FAMOUS BURGERS AND FRIES,Manhattan,2847,BROADFWAY,NaN,2126787701,Hamburgers,03/01/2019,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructed. Unacceptable material used. Non-food contact su...,Not Critical,3,A,03/01/2019,03/24/2022,Cycle Inspection / Initial Inspection,0.000000,0.000000,NaN,NaN,NaN,NaN,1.000000e+00,NaN,True,False,False
186223,50032876,TANNER SMITH'S,Manhattan,204,WEST 55 STREET,10019.0,9175172283,American,06/07/2019,Violations were cited in the following area(s).,08A,Facility not vermin proof. Harborage or conditions conducive to attracting vermin to the premise...,Not Critical,12,A,06/07/2019,03/24/2022,Cycle Inspection / Initial Inspection,40.764364,-73.981362,105.0,4.0,13700.0,1024857.0,1.010260e+09,MN17,True,False,False
186224,50003842,T- 45,Manhattan,135,WEST 45 STREET,10036.0,6466403775,American,07/26/2018,Violations were cited in the following area(s).,10B,Plumbing not properly installed or maintained; anti-siphonage or backflow prevention device not ...,Not Critical,49,NaN,NaN,03/24/2022,Cycle Inspection / Initial Inspection,40.757244,-73.983815,105.0,4.0,11900.0,1089796.0,1.009980e+09,MN17,False,False,True
186225,41640824,RICURAS ECUADORIAN BAKERY,Bronx,1576,WATSON AVENUE,10472.0,7184508363,Spanish,07/20/2018,Violations were cited in the following area(s).,06F,Wiping cloths soiled or not stored in sanitizing solution.,Critical,12,A,07/20/2018,03/24/2022,Cycle Inspection / Initial Inspection,40.826352,-73.876066,209.0,18.0,5001.0,2023533.0,2.037160e+09,BX55,True,False,False


In [128]:
doh['PHONE'] = '+1'+doh['PHONE']

Of the 19,792 unique restaurants, 12,221 did not pass the intitial inspections at least once.

In [129]:
doh_grades = doh.groupby(['CAMIS'])['PHONE','BORO','BUILDING','STREET','ZIPCODE',
                                    'PHONE','CUISINE DESCRIPTION',
                                    'Latitude','Longitude','Community Board',
                                    'Council District','Census Tract',
                                    'A', 'B','C'].sum()

In [130]:
doh_grades

,ZIPCODE,Latitude,Longitude,Community Board,Council District,Census Tract,A,B,C
CAMIS,,,,,,,,,
30075445,83696.0,326.785850,-590.847775,1688.0,104.0,201600.0,1,7,0
30112340,56125.0,203.313258,-369.810407,1545.0,200.0,163500.0,5,0,0
30191841,50095.0,203.836629,-369.921552,520.0,15.0,69500.0,5,0,0
40356018,44896.0,162.319681,-295.928361,1252.0,188.0,139200.0,4,0,0
40356483,101106.0,365.581004,-665.162905,2862.0,414.0,630000.0,5,4,0
...,...,...,...,...,...,...,...,...,...
50115169,10002.0,40.714841,-73.991700,103.0,1.0,1600.0,1,0,0
50116155,33699.0,122.051758,-221.752145,948.0,123.0,111900.0,3,0,0
50117350,50090.0,203.762660,-369.947575,525.0,15.0,54500.0,5,0,0


In [133]:
doh_grades[doh_grades['B'] > 0]

,ZIPCODE,Latitude,Longitude,Community Board,Council District,Census Tract,A,B,C
CAMIS,,,,,,,,,
30075445,83696.0,326.785850,-590.847775,1688.0,104.0,201600.0,1,7,0
40356483,101106.0,365.581004,-665.162905,2862.0,414.0,630000.0,5,4,0
40362264,70175.0,285.548344,-517.773967,749.0,42.0,126700.0,3,4,0
40362274,70084.0,285.080205,-517.982347,714.0,7.0,38507.0,1,6,0
40362715,230115.0,936.235858,-1702.198016,2323.0,23.0,16100.0,7,11,5
...,...,...,...,...,...,...,...,...,...
50111805,30042.0,122.195939,-222.011203,306.0,9.0,20100.0,0,3,0
50113198,40008.0,162.854183,-295.984590,412.0,4.0,3200.0,0,4,0
50113951,41820.0,163.249463,-295.613108,808.0,32.0,33200.0,0,4,0


Of the 19,792 unique restaurants, 9,978 failed an initial cycle inspection at least once.

In [132]:
doh_grades[doh_grades['C'] > 0]

,ZIPCODE,Latitude,Longitude,Community Board,Council District,Census Tract,A,B,C
CAMIS,,,,,,,,,
40360076,112180.0,406.433297,-739.781887,3120.0,390.0,496000.0,4,0,6
40362715,230115.0,936.235858,-1702.198016,2323.0,23.0,16100.0,7,11,5
40363630,200560.0,815.505622,-1479.131518,2160.0,100.0,280000.0,2,11,7
40364179,190475.0,775.226054,-1405.243039,2033.0,133.0,366700.0,3,5,11
40364362,120228.0,489.124894,-887.733038,1260.0,48.0,124800.0,3,3,6
...,...,...,...,...,...,...,...,...,...
50110216,79604.0,285.233461,-517.186091,2828.0,147.0,188307.0,0,0,7
50112974,70049.0,285.004311,-518.044896,707.0,7.0,14700.0,0,0,7
50113139,100926.0,365.433842,-665.981428,2799.0,423.0,259200.0,0,0,9


Of the 19,792 unique restaurants, 5,648 severly failed an initial cycle inspection at least once and are at risk of being closed by the DOHMH.

In [134]:
doh.duplicated().sum()

11502

In [135]:
doh.drop_duplicates(inplace=True)

In [136]:
doh.shape

(174712, 29)

In [137]:
doh['SCORE'].isna().sum()

0

In [ ]:
doh['SCORE'].hist(bins=113, figsize=(12,8));

In [ ]:
doh['SCORE'].value_counts()

In [ ]:
doh['SCORE'].mean()

In [ ]:
doh['SCORE'].median()

In [ ]:
doh['SCORE'].mode()

In [ ]:
doh['GRADE'].isna().sum()

In [ ]:
doh['GRADE'].value_counts()

In [ ]:
doh.info()

In [ ]:
doh['INSPECTION DATE'] =  pd.to_datetime(doh['INSPECTION DATE'])

In [ ]:
# doh.pivot(columns=['CAMIS','INSPECTION DATE'],values='SCORE')

The New York Health Department inspects the approximately 27,000 restaurants within the city to monitor their compliance with food safety regulations. Inspectors observe how food is prepared, served and stored and whether restaurant workers are practicing good hygiene. They check food temperatures, equipment maintenance and pest control measures.

Calling API

In [ ]:
dohmh_df['score'].hist(bins='auto', figsize=(12,8));

In [ ]:
# # Take a look at duplicated records
# duplicates = df1.duplicated(subset=['camis'], keep=False)
# df1.loc[duplicates.loc[duplicates==True].index].sort_values(by='camis')

## Yelp API

In [8]:
with open('/Users/Rob/.secret/yelp_api.json') as f:
    creds = json.load(f)

In [9]:
creds.keys()

dict_keys(['api_key'])

In [10]:
#Business Search      
# url = 'https://api.yelp.com/v3/businesses/search'

#Business Match       
# url = 'https://api.yelp.com/v3/businesses/matches'

#Phone Search         
url = 'https://api.yelp.com/v3/businesses/search/phone'

#Business Details     
# url = 'https://api.yelp.com/v3/businesses/{id}'

#Business Reviews     
# url = 'https://api.yelp.com/v3/businesses/{id}/reviews'

In [ ]:
# #Business Search  
# headers = {
#     'Authorization': 'Bearer ' + creds['api_key']
#           }
# location = 'New York NY'
# SEARCH_LIMIT = 50

# url_params = {
#     'limit': SEARCH_LIMIT,
#     'location' : location.replace(' ','+'),
#     'offset': 0
#              }


# response = requests.get(url, headers=headers, params=url_params)
# print(response.status_code)

In [140]:
phones = list(top_doh['PHONE'])
phones

['+12129569488',
 '+17187087600',
 '+12128882288',
 '+12128882288',
 '+17182719222']

In [73]:
phone_numbers = set(doh['PHONE'])

In [74]:
len(phone_numbers)

18938

In [154]:
phone_numbers = list(phone_numbers)

In [155]:
phone_numbers1 = phone_numbers[1:1000]

In [199]:
phone_numbers2 = phone_numbers[1000:2000]

In [235]:
phone_numbers3 = phone_numbers[2000:2500]

In [236]:
phone_numbers4 = phone_numbers[2500:3500]

In [254]:
phone_numbers5 = phone_numbers[3500:5000]

In [298]:
phone_numbers6 = phone_numbers[5000:6000]

In [306]:
phone_numbers7 = phone_numbers[6000:7500]

In [307]:
# Initialize empty list
biz_list7 = []

In [308]:
#Phone Search 
for number in phone_numbers7:
    url = 'https://api.yelp.com/v3/businesses/search/phone'
    headers = {'Authorization': 'Bearer ' + creds['api_key']}
    url_params = {'phone': number}
    response = requests.get(url, headers=headers, params=url_params)
    response_json = response.json()
    biz_list7.extend(response_json.get('businesses','U'))
    

In [309]:
while 'U' in biz_list7:
    biz_list7.remove('U')

In [310]:
biz_list7_df = pd.DataFrame(biz_list7)
biz_list7_df.to_csv('yelp_phone7.csv',index=False)
df = pd.read_csv('yelp_phone7.csv')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone
0,WCiGrGn89yBrVbqXRkejwg,jamaican-flavors-jamaica,Jamaican Flavors,https://s3-media0.fl.yelpcdn.com/bphoto/GdYOODcwJLYZGd_C231qcw/o.jpg,False,https://www.yelp.com/biz/jamaican-flavors-jamaica?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_cam...,111,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {'alias': 'soulfood', 'title': 'Soul Food'}, {'alia...",4.0,"{'latitude': 40.705629, 'longitude': -73.795176}",[],$,"{'address1': '16417 Jamaica Ave', 'address2': '', 'address3': '', 'city': 'Jamaica', 'zip_code':...",17185262228,(718) 526-2228
1,vnf1WSV7dTmOeOkIJ46hiw,curry-king-new-york,Curry King,https://s3-media0.fl.yelpcdn.com/bphoto/dN8o-aaavsnCvjgvZDXhsw/o.jpg,False,https://www.yelp.com/biz/curry-king-new-york?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campaign...,68,"[{'alias': 'indpak', 'title': 'Indian'}, {'alias': 'halal', 'title': 'Halal'}, {'alias': 'pakist...",2.5,"{'latitude': 40.799566279368, 'longitude': -73.9629824840441}","['pickup', 'delivery']",$,"{'address1': '942 Columbus Ave', 'address2': '', 'address3': '', 'city': 'New York', 'zip_code':...",16466697826,(646) 669-7826
2,0sFAy3tueTdZYIywI_87AA,paris-baguette-flushing-4,Paris Baguette,https://s3-media0.fl.yelpcdn.com/bphoto/M3prfsZvyyiRVsWfNm3_yg/o.jpg,False,https://www.yelp.com/biz/paris-baguette-flushing-4?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_ca...,194,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias': 'bakeries', 'title': 'Bakeries'}, {'ali...",3.5,"{'latitude': 40.7635157983811, 'longitude': -73.8088456168771}",['delivery'],$$,"{'address1': '156-24 Northern Blvd', 'address2': '', 'address3': '', 'city': 'Flushing', 'zip_co...",17189610404,(718) 961-0404
3,-OplSkpDk33uikS5fO-JQw,addictive-wine-and-tapas-bar-jackson-heights-4,Addictive Wine and Tapas Bar,https://s3-media0.fl.yelpcdn.com/bphoto/n1_yRJ1iQEuXFaerOTy3-Q/o.jpg,False,https://www.yelp.com/biz/addictive-wine-and-tapas-bar-jackson-heights-4?adjust_creative=82uXkAt1...,231,"[{'alias': 'wine_bars', 'title': 'Wine Bars'}, {'alias': 'tapas', 'title': 'Tapas Bars'}, {'alia...",4.5,"{'latitude': 40.75632, 'longitude': -73.8806}","['pickup', 'delivery']",$$,"{'address1': '87-01 Northern Blvd', 'address2': None, 'address3': '', 'city': 'Jackson Heights',...",13478320951,(347) 832-0951
4,ePZjA9S2xD6ZKpTfaH3Irg,gong-cha-brooklyn,Gong Cha,https://s3-media0.fl.yelpcdn.com/bphoto/4lfIopMiXui_BOcSdGwTbQ/o.jpg,False,https://www.yelp.com/biz/gong-cha-brooklyn?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campaign=y...,60,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}]",4.0,"{'latitude': 40.69147, 'longitude': -73.98643}","['delivery', 'pickup']",$,"{'address1': '156 Lawrence St', 'address2': None, 'address3': '', 'city': 'Brooklyn', 'zip_code'...",19179091207,(917) 909-1207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,TynT1ykBkQhXd6nuTK6JUQ,rego-bagels-rego-park-2,Rego Bagels,https://s3-media0.fl.yelpcdn.com/bphoto/kwLN-ldWtcuj5Y7YfXafqA/o.jpg,False,https://www.yelp.com/biz/rego-bagels-rego-park-2?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_camp...,171,"[{'alias': 'bagels', 'title': 'Bagels'}, {'alias': 'coffee', 'title': 'Coffee & Tea'}, {'alias':...",4.0,"{'latitude': 40.7295312415351, 'longitude': -73.8625347233294}","['pickup', 'delivery']",$,"{'address1': '9508 63rd Dr', 'address2': '', 'address3': '', 'city': 'Rego Park', 'zip_code': '1...",17184596265,(718) 459-6265
1270,mUl0wtO7tPkBocuJqZ-7Dw,dun-huang-grand-central-new-york-4,Dun Huang - Grand Central,https://s3-media0.fl.yelpcdn.com/bphoto/TmMr0NikR2kI8TXBHZ7qJw/o.jpg,False,https://www.yelp.com/biz/dun-huang-grand-central-new-york-4?adjust_creative=82uXkAt1Tiw7u9_h33zr...,245,"[{'alias': 'chinese', 'title': 'Chinese'}, {'alias': 'noodles', 'title': 'Noodles'}]",4.5,"{'latitude': 40.74922, 'longitude': -73.97785}","['delivery', 'pickup']",$$,"{'address1': '320 Lexington Ave', 'address2': None, 'address3

In [221]:
df_1 = pd.read_csv('yelp_phone1.csv')

In [223]:
df_2 = pd.read_csv('yelp_phone2.csv')

In [229]:
df_3 = pd.read_csv('yelp_phone3.csv')

In [230]:
yelp_df = pd.concat([df_1,df_2,df_3],ignore_index=True)

In [231]:
yelp_df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone
0,fEsgUESZxOQtd4YEmAUuow,ii-caffe-latte-2-new-york-4,II Caffe Latte 2,https://s3-media0.fl.yelpcdn.com/bphoto/rL-1bjIrEvnG6zV8ofrZpg/o.jpg,False,https://www.yelp.com/biz/ii-caffe-latte-2-new-york-4?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_...,127,"[{'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}, {'alias': 'desserts', 'title': 'D...",3.5,"{'latitude': 40.8246899, 'longitude': -73.94704}","['delivery', 'pickup']",$$,"{'address1': '458 W 145th St', 'address2': '', 'address3': '', 'city': 'New York', 'zip_code': '...",12122340202,(212) 234-0202
1,zm_ngfMxyU6d85AaTIgI7Q,chloes-restaurant-and-lounge-brooklyn,Chloe's Restaurant & Lounge,https://s3-media0.fl.yelpcdn.com/bphoto/AFD_nPVpU1rhI7VMWvbltA/o.jpg,False,https://www.yelp.com/biz/chloes-restaurant-and-lounge-brooklyn?adjust_creative=82uXkAt1Tiw7u9_h3...,192,"[{'alias': 'haitian', 'title': 'Haitian'}, {'alias': 'salad', 'title': 'Salad'}, {'alias': 'seaf...",2.5,"{'latitude': 40.63755, 'longitude': -73.89731}","['delivery', 'pickup']",$$,"{'address1': '9413 Ave L', 'address2': None, 'address3': '', 'city': 'Brooklyn', 'zip_code': '11...",13477709051,(347) 770-9051
2,LbGAKVX3cwZj4fUC2XOLxg,jahns-jackson-heights,Jahn's,https://s3-media0.fl.yelpcdn.com/bphoto/9Oa1n8Dyo_n2N9YIhJxsgw/o.jpg,False,https://www.yelp.com/biz/jahns-jackson-heights?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campai...,172,"[{'alias': 'diners', 'title': 'Diners'}]",3.5,"{'latitude': 40.74941, 'longitude': -73.88497}","['delivery', 'pickup']",$$,"{'address1': '81-04 37th Ave', 'address2': '', 'address3': '', 'city': 'Jackson Heights', 'zip_c...",17186510700,(718) 651-0700
3,i8ejDDR4COtukAAA1Ls5fw,soba-ya-new-york,Soba-ya,https://s3-media0.fl.yelpcdn.com/bphoto/LnjY25INn-aM1eAyiav8XA/o.jpg,False,https://www.yelp.com/biz/soba-ya-new-york?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campaign=ye...,1496,"[{'alias': 'japanese', 'title': 'Japanese'}, {'alias': 'noodles', 'title': 'Noodles'}]",4.0,"{'latitude': 40.72965, 'longitude': -73.9878899}","['delivery', 'pickup']",$$,"{'address1': '229 E 9th St', 'address2': '', 'address3': '', 'city': 'New York', 'zip_code': '10...",12125336966,(212) 533-6966
4,YV0VaJgV2zotzI7XNUaMCQ,mcdonalds-new-york-100,McDonald's,https://s3-media0.fl.yelpcdn.com/bphoto/X710tzNcQt5xzpJMVIpZHw/o.jpg,False,https://www.yelp.com/biz/mcdonalds-new-york-100?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campa...,85,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {'alias': 'burgers', 'title': 'Burgers'}, {'alias':...",2.5,"{'latitude': 40.71852, 'longitude': -74.00115}",['delivery'],$,"{'address1': '262 Canal St', 'address2': None, 'address3': '', 'city': 'New York', 'zip_code': '...",12129415823,(212) 941-5823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,aIRhO0WZ3KprUphL3oYsBg,applebees-grill-bar-staten-island-4,Applebee's Grill + Bar,https://s3-media0.fl.yelpcdn.com/bphoto/oIR3BQHVVgdXBWCS7bAbJg/o.jpg,False,https://www.yelp.com/biz/applebees-grill-bar-staten-island-4?adjust_creative=82uXkAt1Tiw7u9_h33z...,67,"[{'alias': 'tradamerican', 'title': 'American (Traditional)'}, {'alias': 'sportsbars', 'title': ...",2.5,"{'latitude': 40.530092, 'longitude': -74.229161}","['delivery', 'pickup']",$$,"{'address1': ""2720 Veteran's Rd W"", 'address2': None, 'address3': None, 'city': 'Staten Island',...",17189431200,(718) 943-1200
2160,-wGvjd9NmPqrT0tm_pq1QQ,the-black-whale-bronx,The Black Whale,https://s3-media0.fl.yelpcdn.com/bphoto/X1ThAksqctFuD5kBszAz7w/o.jpg,False,https://www.yelp.com/biz/the-black-whale-bronx?adjust_creative=82uXkAt1Tiw7u9_h33zr1A&utm_campai...,184,"[{'alias': 'newamerican', 'title': 'American (New)'}, {'alias': 'diners', 'title': 'Diners'}, {'...",3.5,"{'latitude': 40.846302, 'longitude': -73.786432}",[],$$,"{'address1': '279 City Island Ave', 'address2': '', 'address3': '', 'city': 'Bronx', 'zip_code':...",17188853657,(718)

In [ ]:
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
#         for key in ("name", "review_count", "rating", "price"):
#             prepared_data[key] = business_data.get(key, None)
    
#         # Parse and add latitude and longitude columns
#         coordinates = business_data["coordinates"]
#         prepared_data["latitude"] = coordinates["latitude"]
#         prepared_data["longitude"] = coordinates["longitude"]
        
#         # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
#     return results
    
# # Test out function
# prepared_businesses = prepare_data(businesses)
# prepared_businesses[:5]

In [ ]:
# full_dataset = []

# for offset in get_offsets(total):
    
#     url_params['offset'] = offset
    
#     response = requests.get(url, headers=headers, params=url_params)
    
#     time.sleep(1)
    
#     response_json = response.json()
    
#     businesses = response_json.get('businesses')
# #    
#     prepared_business = prepare_data(businesses)
    
#     full_dataset.extend(prepared_businesses)

# len(full_dataset)

In [ ]:
# # Loop to retrieve all the results from the original request for all NYC businesses
# offset=0
# response = get_results('New York,NY',offset=offset)

# n_per_page = len(response['businesses'])
# total_result = response['total']

# results = response_json.get('businesses',{})

# while offset < total_result:
#     offset+=n_per_page
#     res = get_results('New York,NY',offset=offset)
#     results.extend(res['businesses'])
# results[:5]

In [ ]:
# lat = []
# long = []

# for _,business in yelp_df.iterrows():
#     lat.append(business['coordinates']['latitude'])
#     long.append(business['coordinates']['longitude'])

# yelp_df['lat'] = lat
# yelp_df['long'] = long

# Data Understanding

For this project there will be two sources and types of data used:

* Historical health and hygiene inspections recorded by New York City Department of Health and Mental Hygiene (DOHMH) public health inspectors
* User generated Yelp business ratings and reviews

## Understanding NYC DOHMH Data

This dataset contains over 330,000 records, let's explore its contents.

In [ ]:
dohmh_df['inspection_date'] =  pd.to_datetime(dohmh_inspections['inspection_date'])

In [ ]:
dohmh_df['inspection_date'].min()

In [ ]:
dohmh_df['inspection_date'].max()

Inspections in this dataset range from May 2008 up to present day.

In [ ]:
# dohmh_df.set_index('inspection_date')

In [ ]:
dohmh_df['boro'].value_counts()

In [ ]:
dohmh_df['score'].value_counts()

In [ ]:
dohmh_df['score'].isna().sum()

In [ ]:
dohmh_df['grade'].value_counts()

In [ ]:
dohmh_df['grade'].value_counts(normalize=True)

In [ ]:
dohmh_df['grade'].isna().sum()

In [ ]:
dohmh_df['critical_flag'].value_counts()

Critical violations are those most likely to contribute to foodborne illness

In [ ]:
dohmh_df['critical_flag'].value_counts(normalize=True)

In [ ]:
dohmh_df['inspection_type'].value_counts()

In [ ]:
dohmh_inspections = dohmh_df[(dohmh_df['inspection_type'] ==
                              'Cycle Inspection / Initial Inspection') | (dohmh_df['inspection_type'] ==
                              'Cycle Inspection / Re-inspection')]

In [ ]:
dohmh_inspections['critical_flag'].value_counts()

In [ ]:
# Drop records with NA critical_flag

dohmh_ohe = dohmh_inspections[dohmh_inspections['critical_flag'] != 'Not Applicable']

In [ ]:
dohmh_ohe = pd.get_dummies(dohmh_inspections,columns=['critical_flag'])

In [ ]:
dohmh_ohe.head(2)

In [ ]:
critical_flags = dohmh_ohe.groupby(['camis'])['critical_flag_Critical'].count()

In [ ]:
critical_flags[critical_flagss]

In [ ]:
critical_flags['critical_flag_Not Critical'].sum()

In [ ]:
non_critical_flags = dohmh_ohe.groupby(['camis'])

In [ ]:
dohmh_inspections.duplicated().sum()

In [ ]:
dohmh_inspections.loc[dohmh_inspections.duplicated(keep='first'),:]

In [ ]:
dohmh_inspections.drop_duplicates().shape

In [ ]:
dohmh_inspections.duplicated(subset=['camis']).sum()

In [ ]:
dohmh_inspections['camis'].nunique()

There are approximately 28,000 restaurants that have been inspected by the DOHMH. Let's check out how many have ever been flagged for a critical violation.

## Mapping with Follium